In [1]:
import numpy as np
import pandas as pd
import os
import tqdm
import torch

from itertools import chain

import matplotlib.pyplot as plt
%matplotlib inline

In [49]:
import sys

sys.path.append('..')

import utils

from importlib import reload

reload(utils)

from utils import dataset, applier, domain, pipe

reload(dataset)
reload(applier)
reload(domain)
reload(pipe)

from utils.dataset import LabeledDataset

In [3]:
applier.mobilenet_v3_small

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU(inplace=True)
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (2): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_r

In [4]:
data_path = "/Users/sergmiller/Documents/my/IDAO-2021/data/idao_dataset/"

In [5]:
train_path = os.path.join(data_path, "train")

In [6]:
sample_dataset = dataset.build_dataset(train_path, limit=10)

100%|██████████| 10/10 [00:00<00:00, 129.92it/s]


In [10]:
sample_dataset.save('test.npz')
sample_dataset = dataset.LabeledDataset.load('test.npz')

In [7]:
sample_dataset.samples.shape

(10, 576, 576)

In [11]:
import torchvision.models as models
import torchvision.transforms as transforms
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# model = models.mobilenet_v3_small(pretrained=True)

In [13]:
torch.set_num_threads(1)

In [14]:
torch.get_num_threads()

1

In [30]:
croped_mobilenet = torch.nn.Sequential(
    transforms.CenterCrop(64),
    applier.mobilenet_v3_small
)

In [31]:
croped_mobilenet.train(False)

Sequential(
  (0): CenterCrop(size=(64, 64))
  (1): MobileNetV3(
    (features): Sequential(
      (0): ConvBNActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
            (relu): ReLU(inplace=True)
            (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          )
          (2): ConvBNActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1

In [8]:
train_dataset = dataset.build_dataset(train_path)

100%|██████████| 13404/13404 [01:37<00:00, 136.97it/s]


In [32]:
train_emb_dataset = applier.build_embd_dataset(train_dataset, croped_mobilenet)

100%|██████████| 838/838 [01:01<00:00, 13.69it/s]


In [82]:
train_emb_dataset.save('../data/center_crop_64_mobilenet_v3_small_embeddings.npz')

PicklingError: Can't pickle <class 'utils.dataset.LabeledDataset'>: it's not the same object as utils.dataset.LabeledDataset

In [154]:
train_emb_dataset = LabeledDataset.load('../data/mobilenet_v3_small_embeddings.npz')

In [34]:
train_emb_dataset.samples.shape

(13404, 1000)

In [35]:
def apply_random_projection(data : np.array, to_dim : int = 10, seed : int = 0):
    assert 2 == len(data.shape)
    np.random.seed(seed)
    N = data.shape[1]
    proj_matrix = np.random.randn(N, to_dim) / np.sqrt(N)
    print(np.mean(proj_matrix), np.std(proj_matrix))
    data_proj = np.matmul(data, proj_matrix)
    return data_proj

In [37]:
import copy
proj_emb_dataset = copy.copy(train_emb_dataset)
proj_emb_dataset.samples = apply_random_projection(
    train_emb_dataset.samples, to_dim=50)

-0.00011997710679547996 0.03151695476906101


In [38]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score

In [39]:
params = {
    "penalty": 'l1',
    "solver": 'saga',
    "verbose": 10,
    "max_iter": 100,
    "tol": 1e-5,
    "scoring" : 'roc_auc',
    "random_state": 0,
    "cv": 10,
    "n_jobs": 7,
    "refit": True
}

In [149]:
ids = np.arange(len(train_proj_emb_dataset.samples))
np.random.seed(0)
np.random.shuffle(ids)
N = int(len(ids) * 0.9)
train_ids = ids[:N]
val_ids = ids[N:]
print(train_ids[:10], val_ids[:10])

[10869  3245 13299 10980  1103  5091 10134  6859  4516  4422] [ 1057 10013  6979  7505    60  6173  4878    35 11686  7315]


In [164]:
train_proj_emb_dataset = proj_emb_dataset.subset(train_ids)
val_proj_emb_dataset = proj_emb_dataset.subset(val_ids)

In [40]:
cv_label1 = LogisticRegressionCV(**params).fit(
    X=proj_emb_dataset.samples,
    y=(proj_emb_dataset.labels[:, 0] == 'NR').astype(int))

[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.


convergence after 4 epochs took 0 seconds
convergence after 4 epochs took 0 seconds
convergence after 4 epochs took 0 secondsconvergence after 4 epochs took 0 seconds
convergence after 4 epochs took 0 seconds

convergence after 4 epochs took 0 seconds
convergence after 4 epochs took 0 seconds
convergence after 38 epochs took 0 seconds
convergence after 38 epochs took 0 seconds
convergence after 38 epochs took 0 seconds
convergence after 39 epochs took 0 seconds
convergence after 38 epochs took 0 seconds
convergence after 39 epochs took 0 seconds
convergence after 39 epochs took 0 seconds
max_iter reached after 2 seconds
max_iter reached after 2 seconds
max_iter reached after 2 seconds
max_iter reached after 2 seconds
max_iter reached after 2 seconds
max_iter reached after 2 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

max_iter reached after 2 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


...

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 0 seconds
max_iter reached after 1 seconds
max_iter reached after 1 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:   22.9s finished


max_iter reached after 1 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [43]:
np.mean(cv_label1.scores_[1],axis=0)  # with crop64

array([0.5       , 0.78958939, 0.93599517, 0.96792367, 0.97494673,
       0.97596747, 0.97633959, 0.97650932, 0.97657727, 0.97661092])

In [127]:
np.mean(cv_label1.scores_[1], axis=0)  # new

array([0.5       , 0.5       , 0.76982608, 0.86866077, 0.9326519 ,
       0.94429291, 0.94827148, 0.95034892, 0.95169425, 0.9526548 ])

In [124]:
np.mean(cv_label1.scores_[1], axis=0)  # old

array([0.5       , 0.5       , 0.7952889 , 0.87402804, 0.94469283,
       0.95739159, 0.96157609, 0.96381906, 0.96529288, 0.96639186])

In [44]:
cv_label1.get_params()

{'Cs': 10,
 'class_weight': None,
 'cv': 10,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1.0,
 'l1_ratios': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': 7,
 'penalty': 'l1',
 'random_state': 0,
 'refit': True,
 'scoring': 'roc_auc',
 'solver': 'saga',
 'tol': 1e-05,
 'verbose': 10}

In [45]:
import copy

classes = [1, 10, 20, 3, 30, 6]

def mae_scorer(estimator, X, y):
    y_pred = np.array([classes[int(_y)] for _y in estimator.predict(X)])
    y = np.array([classes[_y] for _y in y])
    return -np.mean(np.abs(y_pred - y))

params2 = copy.copy(params)

params2['scoring'] = mae_scorer
# params2['class_weight'] = {str(c) : c for c in classes}
cv_label2 = LogisticRegressionCV(**params2).fit(
    proj_emb_dataset.samples,
    proj_emb_dataset.labels[:, 1])

[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.


convergence after 4 epochs took 0 seconds
convergence after 4 epochs took 0 secondsconvergence after 4 epochs took 0 seconds

convergence after 4 epochs took 0 secondsconvergence after 4 epochs took 0 seconds

convergence after 4 epochs took 0 seconds
convergence after 6 epochs took 0 seconds
convergence after 94 epochs took 4 seconds
convergence after 94 epochs took 4 seconds
convergence after 94 epochs took 4 seconds
convergence after 94 epochs took 4 seconds
convergence after 94 epochs took 4 seconds
convergence after 94 epochs took 4 seconds
convergence after 94 epochs took 4 seconds
max_iter reached after 4 seconds
max_iter reached after 5 seconds
max_iter reached after 5 seconds
max_iter reached after 5 seconds
max_iter reached after 5 seconds
max_iter reached after 5 seconds
max_iter reached after 5 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

max_iter reached after 4 seconds
max_iter reached after 5 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

...

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 3 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:  1.2min finished


max_iter reached after 3 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [46]:
np.mean(cv_label2.scores_['1'],axis=0) # crop64 proj

array([-8.3034167 , -2.87024781, -1.16681787, -0.68667234, -0.50216841,
       -0.46188559, -0.43882784, -0.42890286, -0.42718655, -0.43352677])

In [24]:
np.mean(cv_label2.scores_['1'],axis=0) # mini weighted

array([7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.7, 5.7, 5.7, 5.7])

In [130]:
np.mean(cv_label2.scores_['1'],axis=0) # not weighted proj

array([-9.31475597, -8.29894242, -2.84744388, -1.16951128, -0.74142058,
       -0.62891488, -0.60055984, -0.58071633, -0.56407921, -0.55960338])

In [142]:
np.mean(cv_label2.scores_['1'],axis=0) # not weighted

array([10.68017027, 10.68017027, 10.09729052,  9.80483741,  9.63690119,
        9.57527796,  9.54491318,  9.53208126,  9.52514296,  9.51149031])

In [144]:
np.mean(cv_label2.scores_['1'],axis=0) # weighted

array([ 9.59965186,  9.4570564 ,  9.96051226, 10.08321655,  9.88160343,
        9.84861552,  9.80512292,  9.85751204,  9.71321238,  9.77365697])

In [47]:
pub_test_dataset = dataset.build_dataset(
    os.path.join(data_path, "public_test"),
    sample_processor=domain.process_test_sample)

100%|██████████| 1502/1502 [00:10<00:00, 140.18it/s]


In [48]:
private_test_dataset = dataset.build_dataset(
    os.path.join(data_path, "private_test"),
    sample_processor=domain.process_test_sample)

100%|██████████| 15062/15062 [01:50<00:00, 135.94it/s]


In [50]:
class ModelWithProjection:
    def __init__(self, model):
        self.model = model
    def predict(self, x):
        x_proj = apply_random_projection(x, to_dim=50)
        return self.model.predict(x_proj)
    def predict_proba(self, x):
        x_proj = apply_random_projection(x, to_dim=50)
        return self.model.predict_proba(x_proj)

In [51]:
model1 = ModelWithProjection(cv_label1)
model2 = ModelWithProjection(cv_label2)

In [52]:
sample_res = pipe.apply_all_models_to_test_dataset(
    sample_dataset,
    croped_mobilenet,
    model1,
    model2,
    'sample')

100%|██████████| 1/1 [00:00<00:00, 913.00it/s]


-0.00011997710679547996 0.03151695476906101
-0.00011997710679547996 0.03151695476906101


In [53]:
sample_res.tags.shape, sample_res.samples.shape

((10,), (10, 1000))

In [54]:
pub_test_predictions = pipe.apply_all_models_to_test_dataset(
    pub_test_dataset, croped_mobilenet, model1, model2, 'pub_test')

100%|██████████| 94/94 [00:08<00:00, 11.25it/s]


-0.00011997710679547996 0.03151695476906101
-0.00011997710679547996 0.03151695476906101


In [55]:
private_test_predictions = pipe.apply_all_models_to_test_dataset(
    private_test_dataset, croped_mobilenet, model1, model2, 'private_test')

100%|██████████| 942/942 [01:28<00:00, 10.63it/s]


-0.00011997710679547996 0.03151695476906101
-0.00011997710679547996 0.03151695476906101


In [81]:
pub_test_predictions.save('../data/pub_test_predictions_embeds_crop_64_proj_50.npz')
private_test_predictions.save('../data/private_test_predictions_embeds_crop_64_proj_50.npz')

In [76]:
pub_test_predictions.labels[:20]

array([['0.5368004206006665', '3'],
       ['0.9998872493536303', '30'],
       ['0.9996172025197383', '30'],
       ['0.9999990579134285', '30'],
       ['0.07784210090765609', '6'],
       ['0.02124472220801743', '20'],
       ['0.9256123002882367', '30'],
       ['0.00040788086824705694', '20'],
       ['0.9922667673856572', '3'],
       ['0.008025452674042644', '1'],
       ['0.5299681165404868', '3'],
       ['0.021550938556567645', '1'],
       ['0.9999999994823845', '30'],
       ['0.9889214166023548', '3'],
       ['0.9999999978258628', '30'],
       ['0.007616206466127817', '20'],
       ['0.000282985850604045', '20'],
       ['0.8565053142065482', '3'],
       ['0.48568883143904296', '3'],
       ['0.9390134451989677', '3']], dtype='<U32')

In [77]:
private_test_predictions.labels[:20]

array([['0.9998074503262109', '10'],
       ['0.07660731889437267', '20'],
       ['0.014631353598697716', '1'],
       ['1.826171939323107e-05', '20'],
       ['0.04586276823393909', '1'],
       ['0.2588877858245321', '3'],
       ['0.9999999999999989', '30'],
       ['0.9890979642274442', '3'],
       ['0.9796929482751457', '10'],
       ['0.7334493212675004', '3'],
       ['0.04853270111317043', '1'],
       ['0.4064774152593026', '3'],
       ['1.732103449558764e-08', '20'],
       ['0.21699602121736916', '6'],
       ['0.1687102849848715', '20'],
       ['0.0006089176937635887', '20'],
       ['0.999997781345173', '30'],
       ['5.387432659098579e-06', '20'],
       ['9.235629913062482e-07', '20'],
       ['0.010071644433182647', '1']], dtype='<U32')

In [70]:
train_emb_dataset.labels[7]

array(['NR', '20'], dtype='<U21')

In [71]:
model1.predict_proba(train_emb_dataset.samples[4].reshape(1, -1))

-0.00011997710679547996 0.03151695476906101


array([[0.03249416, 0.96750584]])

In [64]:
model2.predict_proba(train_emb_dataset.samples[2].reshape(1, -1))

-0.00011997710679547996 0.03151695476906101


array([[1.65388373e-22, 1.10186400e-02, 9.88966091e-01, 1.22186016e-11,
        1.52692594e-05, 6.21623352e-11]])

In [72]:
submit = dataset.LabeledDataset.merge(
    pub_test_predictions, private_test_predictions)

In [198]:
submit.labels[:, 0] = np.array([1 - float(x) for x in submit.labels[:, 0]])

In [73]:
submit.labels[:10]

array([['0.5368004206006665', '3'],
       ['0.9998872493536303', '30'],
       ['0.9996172025197383', '30'],
       ['0.9999990579134285', '30'],
       ['0.07784210090765609', '6'],
       ['0.02124472220801743', '20'],
       ['0.9256123002882367', '30'],
       ['0.00040788086824705694', '20'],
       ['0.9922667673856572', '3'],
       ['0.008025452674042644', '1']], dtype='<U32')

In [78]:
dataset.dataset2submit_csv(submit,
    "../submits/1:log_reg_2:log_reg_multiclass_not_weighted_over_proj50_crop_64_seed_0_mobilenet_v3_small_embds.csv")

In [79]:
submit.tags.shape

(16564,)

In [80]:
submit.samples.shape

(16564, 1000)

In [194]:
pub_test_predictions.tags.shape, pub_test_predictions.samples.shape

((1502,), (1502, 1000))

In [195]:
private_test_predictions.tags.shape, private_test_predictions.samples.shape

((15062,), (15062, 1000))

In [244]:
model = applier.mobilenet_v3_small
model.train(False)

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU(inplace=True)
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (2): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_r

In [245]:
torch.save(model.state_dict(), "mobilenet_state_dict")

In [246]:
import torchvision
model2 = mobilenet_v3_small()

In [247]:
model2.load_state_dict(torch.load("mobilenet_state_dict"))

<All keys matched successfully>

In [248]:
np.random.seed(0)
t = np.random.random((1,3,500,500))
model2.train(False)
print(np.mean(t), np.std(t))
with torch.no_grad():
    res = model2(torch.Tensor(t)).detach().numpy()
print(np.mean(res), np.std(res))

0.5002803215266387 0.2887019402063319
0.46069193 1.7000295


In [249]:
np.random.seed(0)
t = np.random.random((1,3,500,500))
model.train(False)
print(np.mean(t), np.std(t))
with torch.no_grad():
    res = model(torch.Tensor(t)).detach().numpy()
print(np.mean(res), np.std(res))

0.5002803215266387 0.2887019402063319
0.46069193 1.7000295


In [214]:
model

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU(inplace=True)
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (2): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_r

In [231]:
import torchvision
mobilenet = torchvision.models.mobilenet_v3_small()
mobilenet.load_state_dict(torch.load("mobilenet_state_dict"))

np.random.seed(0)
t = np.random.random((1,3,500,500))
model2.train(False)
print(np.mean(t), np.std(t))
with torch.no_grad():
    res = mobilenet(torch.Tensor(t)).detach().numpy()
print(np.mean(res), np.std(res))



import time
start = time.time()
for i in np.arange(10):
    t = np.random.random((32,3,500,500))
    with torch.no_grad():
        res = mobilenet(torch.Tensor(t)).detach().numpy()
    
print('OK, run {}s'.format(time.time() - start))

0.5002803215266387 0.2887019402063319
3.4595482 0.81053764
OK, run 25.1478328704834s


In [242]:
import sys

sys.path.append('..')
from submition_track2.test_in_submit.mobilenet_v3 import mobilenet_v3_small

In [243]:
mobilenet = mobilenet_v3_small()